In [47]:
import igraph as ig
import leidenalg as la
import numpy as np
import pandas as pd
import collections
from collections import Counter

import networkx as nx

In [34]:
g = ig.Graph.Read_GraphML("graphall.graphml")

In [49]:
# Define capacity threshold and initialize counter
capacity_threshold = 15000000
count_printed = 0

# Print source and target identifiers, names, and capacities for edges above threshold
for edge in g.es:
    if edge['capacity'] > capacity_threshold:
        source_vertex = g.vs[edge.source]
        target_vertex = g.vs[edge.target]
        print(f"Edge from {source_vertex['alias']} (Source: {source_vertex['name']}) to {target_vertex['alias']} (Target: {target_vertex['name']}) with capacity {edge['capacity']}")
        count_printed += 1
        if count_printed == 10:
            break


Edge from BeerselLightningNode (Source: 3334852914) to LNBϟG [Edge-1] (Target: 3977033294) with capacity 16777215.0
Edge from hodlister_co (Source: 2248511899) to LNBϟG [Edge-1] (Target: 3977033294) with capacity 16777215.0
Edge from satjar (Source: 2120228994) to LNBϟG [Edge-1] (Target: 3977033294) with capacity 16777215.0
Edge from PeerName.com (Source: 1937807929) to LNBϟG [Edge-1] (Target: 3977033294) with capacity 16777215.0
Edge from Johoe (Source: 2183444107) to LNBϟG [Edge-2] (Target: 3311945645) with capacity 16777215.0
Edge from my-node (Source: 653231100) to LNBϟG [Edge-4] (Target: 4021163737) with capacity 16775701.0
Edge from ⚡⚡ No Noobs ⚡⚡ (Source: 3836442376) to LNBϟG [Edge-4] (Target: 4021163737) with capacity 16777215.0
Edge from my-node (Source: 653231100) to LNBϟG [Edge-2] (Target: 3311945645) with capacity 16777215.0
Edge from CoinPlaza.it (Source: 1948046283) to LNBϟG [Hub-3] (Target: 1129663831) with capacity 16777215.0
Edge from CoinPlaza.it (Source: 1948046283) 

In [51]:
# Find the vertex with the alias 'sorukumar'
vertex = g.vs.find(alias='sorukumar')

# Check if vertex was found and print all attributes
if vertex:
    print(f"All attributes for the node with alias 'sorukumar':")
    for attribute, value in vertex.attributes().items():
        print(f"{attribute}: {value}")
else:
    print("No node with the alias 'sorukumar' found.")


All attributes for the node with alias 'sorukumar':
name: 1027534202
alias: sorukumar
Total_Channels: 19.0
log_chnlcnt: 2.99573227355399
Total_Capacity: 62716644.0
Formatted_Total_Capacity: 62.7m sats
Capacity_Rank: 999.0
Channel_Count_Rank: 857.0
Capacity_Percentile: Top 20%
pub_key: 0225c2a44be98ae923118e92c5b58033a0fd7dd3135be99ee1047ec9d30291dc5d
community: 2.0
id: n4876


# Shortest path

In [53]:
source_node = '1027534202'  # Ensure these are strings if names are stored as strings
target_node = '2120228994'

# Convert to string if necessary
source_node_str = str(source_node)
target_node_str = str(target_node)

# Find vertex indices from node names
index_start = g.vs.find(name=source_node_str).index
index_end = g.vs.find(name=target_node_str).index

# Compute shortest path
path = g.get_shortest_paths(index_start, to=index_end, output="vpath")

# Translate indices back to names for easier interpretation
named_path = [g.vs[vertex]['alias'] for vertex in path[0]]

print(f"Shortest path from {source_node} to {target_node} is:")
print({' -> '.join(named_path)})



Shortest path from 1027534202 to 2120228994 is:
{'sorukumar -> ACINQ -> satjar'}


In [55]:
source_node = '1027534202'  # Source node identifier
target_node = '2120228994'  # Target node identifier

# Ensure names are treated as strings if they are stored as strings in the graph
source_node_str = str(source_node)
target_node_str = str(target_node)

# Find vertex indices from node names
try:
    source_index = g.vs.find(name_eq=source_node_str).index
    target_index = g.vs.find(name_eq=target_node_str).index
except ValueError:
    print("One of the node names provided does not exist in the graph.")
    raise

# Compute all shortest paths
all_paths = g.get_all_shortest_paths(source_index, to=target_index)

# Limit to printing only the first 10 paths
max_paths_to_print = 50
print_count = min(len(all_paths), max_paths_to_print)

print(f"Showing up to {print_count} shortest paths from {source_node} to {target_node}:")

for i in range(print_count):
    # Translate vertex indices in the path to node aliases for easier interpretation
    path_aliases = [g.vs[vertex]['alias'] for vertex in all_paths[i]]
    print(f"Path {i+1}: {' -> '.join(path_aliases)}")


Showing up to 4 shortest paths from 1027534202 to 2120228994:
Path 1: sorukumar -> LQwD-Canada -> satjar
Path 2: sorukumar -> Kraken 🐙⚡ -> satjar
Path 3: sorukumar -> WalletOfSatoshi.com -> satjar
Path 4: sorukumar -> ACINQ -> satjar


In [44]:
def find_valid_paths(graph, source_name, target_name, max_length=6, exclude_names=None):
    """
    Find all valid paths between two nodes sorted by path length up to a maximum length,
    excluding specified nodes.
    
    Args:
    - graph (ig.Graph): The graph object.
    - source_name (str): Name of the source node.
    - target_name (str): Name of the target node.
    - max_length (int): Maximum length of paths to consider.
    - exclude_names (list): List of node names to exclude from the paths.
    
    Returns:
    - list: Sorted list of valid paths, where each path is a list of node aliases.
    """
    try:
        source_index = graph.vs.find(name_eq=source_name).index
    except ValueError:
        print(f"Error: Source node '{source_name}' not found in the graph.")
        return []
    
    try:
        target_index = graph.vs.find(name_eq=target_name).index
    except ValueError:
        print(f"Error: Target node '{target_name}' not found in the graph.")
        return []
    
    exclude_indices = set()
    if exclude_names:
        for name in exclude_names:
            try:
                exclude_indices.add(graph.vs.find(name_eq=name).index)
            except ValueError:
                print(f"Warning: Node to exclude '{name}' not found in the graph.")
    
    # Retrieve all paths from source to target
    all_paths = graph.get_all_simple_paths(source_index, target_index)

    # Filter and sort paths based on maximum length and exclusion criteria
    valid_paths = []
    for path in all_paths:
        if len(path) <= max_length and not set(path).intersection(exclude_indices):
            path_aliases = [graph.vs[vertex_id]['alias'] for vertex_id in path]
            valid_paths.append(path_aliases)

    # Sort paths by their length
    valid_paths.sort(key=len)

    if not valid_paths:
        print("No valid paths found meeting the criteria.")
    return valid_paths

# Example usage:
source_name = '1027534202'  
target_name = '2120228994'  
max_length = 4
exclude_names = ['5']

valid_paths = find_valid_paths(g, source_name, target_name, max_length, exclude_names)

# Print paths by increasing length
current_length = 0
for path in valid_paths:
    if len(path) > current_length:
        current_length = len(path)
        print(f"\nPaths of length {current_length}:")
    print(" -> ".join(path))


KeyboardInterrupt: 

In [ ]:

source_name = '1027534202'  
target_name = '2120228994'  
max_length = 4
exclude_names = ['5', '6']


valid_paths = find_valid_paths(g, source_name, target_name, max_length, exclude_names)

# Print paths by increasing length
current_length = 0
for path in valid_paths:
    if len(path) > current_length:
        current_length = len(path)
        print(f"\nPaths of length {current_length}:")
    print(" -> ".join(path))


In [65]:
def find_paths_with_constraints(graph, source_name, target_name, max_length=5, exclude_names=None):
    """
    Find all paths between two nodes up to a specified maximum length using a depth-limited search,
    excluding specified nodes.

    Args:
    - graph (ig.Graph): The graph object.
    - source_name (str): Name of the source node.
    - target_name (str): Name of the target node.
    - max_length (int): Maximum length of paths to explore.
    - exclude_names (list): List of node names to exclude from the paths.

    Returns:
    - list: A list of paths, where each path is represented as a list of node aliases.
    """
    # Convert names to vertex indices and prepare exclusion set
    source_index = graph.vs.find(name_eq=source_name).index
    target_index = graph.vs.find(name_eq=target_name).index
    exclude_indices = {graph.vs.find(name_eq=name).index for name in exclude_names} if exclude_names else set()

    # Inner function to perform DFS up to the given depth, avoiding excluded nodes
    def dfs(current_path, current_length):
        last_vertex = current_path[-1]
        if last_vertex == target_index:
            paths.append(current_path.copy())
            return
        if current_length == max_length:
            return
        
        for neighbor in graph.neighbors(last_vertex, mode="out"):
            if neighbor not in current_path and neighbor not in exclude_indices:  # Avoid cycles and excluded nodes
                current_path.append(neighbor)
                dfs(current_path, current_length + 1)
                current_path.pop()
    
    # Initialize list to hold paths and start DFS from the source vertex
    paths = []
    dfs([source_index], 0)
    
    # Convert paths from indices to aliases for easier reading
    alias_paths = [[graph.vs[index]['alias'] for index in path] for path in paths]
    
    return alias_paths

# Example usage
source_name = '1027534202'  
target_name = '2120228994'  
# exclude_names = ['5', '6']  # Example node names to exclude
paths = find_paths_with_constraints(g, source_name, target_name, max_length=3)



In [67]:
current_length = 0
for path in paths:
    if len(path) > current_length:
        current_length = len(path)
        print(f"\nPaths of length {current_length}:")
    print(" -> ".join(path))


Paths of length 4:
sorukumar -> ACINQ -> CoinGate -> satjar
sorukumar -> ACINQ -> rompert.hashposition.com🔵 -> satjar
sorukumar -> ACINQ -> satjar
sorukumar -> ACINQ -> LNBϟG [Edge-1] -> satjar
sorukumar -> ACINQ -> BitKassa LN34961 [LND] -> satjar
sorukumar -> ACINQ -> Bitrefill Routing -> satjar
sorukumar -> ACINQ -> OpenNode.com -> satjar
sorukumar -> ACINQ -> bfx-lnd0 -> satjar
sorukumar -> ACINQ -> bfx-lnd0 -> satjar
sorukumar -> ACINQ -> bfx-lnd0 -> satjar
sorukumar -> ACINQ -> bfx-lnd0 -> satjar
sorukumar -> ACINQ -> bfx-lnd0 -> satjar
sorukumar -> ACINQ -> bfx-lnd0 -> satjar
sorukumar -> ACINQ -> bfx-lnd0 -> satjar
sorukumar -> ACINQ -> bfx-lnd0 -> satjar
sorukumar -> ACINQ -> blue 🐘 -> satjar
sorukumar -> ACINQ -> Highway to Hell -> satjar
sorukumar -> ACINQ -> bfx-lnd1 -> satjar
sorukumar -> ACINQ -> bfx-lnd1 -> satjar
sorukumar -> ACINQ -> bfx-lnd1 -> satjar
sorukumar -> ACINQ -> bfx-lnd1 -> satjar
sorukumar -> ACINQ -> bfx-lnd1 -> satjar
sorukumar -> ACINQ -> bfx-lnd1 -> s

In [73]:



def convert_igraph_to_networkx_via_adjacency(igraph_graph):
    """
    Convert an igraph graph to a NetworkX graph using the adjacency matrix method.

    Args:
    - igraph_graph (ig.Graph): The igraph graph to convert.

    Returns:
    - nx.Graph: A NetworkX graph representing the same connectivity.
    """
    # Get the adjacency matrix as a numpy array
    adj_matrix = np.array(igraph_graph.get_adjacency().data)
    # Create a NetworkX graph from this adjacency matrix
    nx_graph = nx.from_numpy_array(adj_matrix, create_using=nx.DiGraph() if igraph_graph.is_directed() else nx.Graph())
    
    # Optionally, transfer node attributes
    for idx, vertex in enumerate(igraph_graph.vs):
        for attr in vertex.attributes().keys():
            nx_graph.nodes[idx][attr] = vertex[attr]

    return nx_graph


g_networkx = convert_igraph_to_networkx_via_adjacency(g)



In [91]:
def convert_igraph_to_networkx(igraph_graph):
    """
    Convert an igraph graph to a NetworkX graph, including both node and edge attributes.

    Args:
    - igraph_graph (ig.Graph): The igraph graph to convert.

    Returns:
    - nx.Graph or nx.DiGraph: A NetworkX graph representing the same connectivity and attributes.
    """
    # Determine the type of NetworkX graph to create based on the directedness of the igraph graph
    nx_graph = nx.DiGraph() if igraph_graph.is_directed() else nx.Graph()
    
    # Add nodes with attributes
    for vertex in igraph_graph.vs:
        nx_graph.add_node(vertex.index, **vertex.attributes())
    
    # Add edges with attributes
    for edge in igraph_graph.es:
        nx_graph.add_edge(edge.source, edge.target, **edge.attributes())
    
    return nx_graph


g_networkx = convert_igraph_to_networkx(g)

In [93]:
first_node = next(iter(g_networkx.nodes(data=True)))
first_edge = next(iter(g_networkx.edges(data=True)))
print("First node and its attributes:", first_node)
print("First edge and its attributes:", first_edge)

First node and its attributes: (0, {'name': '3733632067', 'alias': 'heliacal', 'Total_Channels': 1.0, 'log_chnlcnt': 0.693147180559945, 'Total_Capacity': 6789.0, 'Formatted_Total_Capacity': '7k sats', 'Capacity_Rank': 13026.0, 'Channel_Count_Rank': 13030.0, 'Capacity_Percentile': 'Bottom 60%', 'pub_key': '027ccec61f4bf1fafb5156931da6527dc104ec3613dd4f4050161d89dd76ab494c', 'community': 0.0, 'id': 'n0'})
First edge and its attributes: (0, 1, {'Channel_Size_Tier': 'My Way', 'capacity': 6789.0, 'log_capacity': 8.82320622055274, 'Formatted_Capacity': '7k sats'})


In [95]:
# Print node indices and their attribute names in the NetworkX graph
# Grab the first node's attributes to understand the attribute structure
first_node, first_node_attrs = next(iter(g_networkx.nodes(data=True)))

# Print the node index and its attribute names
print(f"First Node ID: {first_node}")
print("Attribute Names:", list(first_node_attrs.keys()))

# Correcting the edge unpacking to include both nodes and the attribute dictionary
(first_edge_node1, first_edge_node2, first_edge_attrs) = next(iter(g_networkx.edges(data=True)))

# Print the edge nodes and attribute names
print(f"First Edge between nodes {first_edge_node1} and {first_edge_node2}")
print("Edge Attribute Names:", list(first_edge_attrs.keys()))



First Node ID: 0
Attribute Names: ['name', 'alias', 'Total_Channels', 'log_chnlcnt', 'Total_Capacity', 'Formatted_Total_Capacity', 'Capacity_Rank', 'Channel_Count_Rank', 'Capacity_Percentile', 'pub_key', 'community', 'id']
First Edge between nodes 0 and 1
Edge Attribute Names: ['Channel_Size_Tier', 'capacity', 'log_capacity', 'Formatted_Capacity']


In [125]:
# Define the alias you're looking for  n4876
# search_alias = 'sorukumar'
# search_alias = 'satjar'  # n27
search_alias = 'WalletOfSatoshi.com' #n636 3157882931
# Search and print attributes for the node with the given alias
node_found = False
for node, attrs in g_networkx.nodes(data=True):
    if attrs.get('alias') == search_alias:
        print(f"All attributes for the node with alias '{search_alias}':")
        for attr, value in attrs.items():
            print(f"{attr}: {value}")
        node_found = True
        break

if not node_found:
    print(f"No node with the alias '{search_alias}' found.")


All attributes for the node with alias 'WalletOfSatoshi.com':
name: 3157882931
alias: WalletOfSatoshi.com
Total_Channels: 1546.0
log_chnlcnt: 7.34407285057307
Total_Capacity: 17744714682.0
Formatted_Total_Capacity: 177.4 bitcoin
Capacity_Rank: 13.0
Channel_Count_Rank: 3.0
Capacity_Percentile: Top 0.5%
pub_key: 035e4ff418fc8b5554c5d9eea66396c227bd429a3251c8cbc711002ba215bfc226
community: 1.0
id: n636


In [ ]:
graph.nodes[source]['id']

In [127]:
def get_node_index_by_attribute(graph, attribute, value):
    """Returns the first node index with the specified attribute value."""
    for node, attrs in graph.nodes(data=True):
        if attrs.get(attribute) == value:
            return node
    return None  # Return None if no match is found

def find_paths_with_constraints_nx(graph, source_name, target_name, max_length=6, exclude_names=None):
    """
    Finds all paths in a NetworkX graph from source to target using node names, with optional constraints on path length and node exclusion.

    Args:
    - graph (nx.Graph): NetworkX graph.
    - source_name (str): Source node identifier by name.
    - target_name (str): Target node identifier by name.
    - max_length (int): Maximum length of paths to consider.
    - exclude_names (list): List of node names to exclude from the paths.

    Returns:
    - list: A list of paths, where each path is a list of node aliases for readability.
    """
    # Retrieve node indices from attributes
    source_index = get_node_index_by_attribute(graph, 'name', source_name)
    target_index = get_node_index_by_attribute(graph, 'name', target_name)
    
    if source_index is None or target_index is None:
        return []  # Return an empty list if source or target is not found

    def is_valid_path(path):
        if len(path) > max_length:  # Path length check
            return False
        if exclude_names and any(graph.nodes[node]['name'] in exclude_names for node in path):
            return False
        return True

    all_paths = []
    for path in nx.all_simple_paths(graph, source=source_index, target=target_index, cutoff=max_length):
        if is_valid_path(path):
            # Convert indices to aliases for output
            alias_path = [graph.nodes[node]['alias'] for node in path]
            all_paths.append(alias_path)

    return all_paths

# Example usage:
source_name = '1027534202'
target_name = '2120228994'
exclude_names = ['3157882931']  # wallet of Satoshis
max_path_length = 3

valid_paths = find_paths_with_constraints_nx(g_networkx, source_name, target_name, max_path_length, exclude_names)
for path in valid_paths:
    print("Valid path:", " -> ".join(path))


Valid path: sorukumar -> ACINQ -> satjar
Valid path: sorukumar -> Kraken 🐙⚡ -> satjar
Valid path: sorukumar -> LQwD-Canada -> satjar


In [123]:
def get_node_index_by_attribute(graph, attribute, value):
    """Returns the first node index with the specified attribute value."""
    for node, attrs in graph.nodes(data=True):
        if attrs.get(attribute) == value:
            return node
    return None  # Return None if no match is found

# Example usage to find paths using aliases or names
source_index = get_node_index_by_attribute(g_networkx, 'name', '1027534202')
target_index = get_node_index_by_attribute(g_networkx, 'name', '2120228994')
max_path_length = 2

if source_index is not None and target_index is not None:
    try:
        simple_paths = nx.all_simple_paths(g_networkx, source=source_index, target=target_index, cutoff=max_path_length)
        path_list = list(simple_paths)
        print(f"Total paths found: {len(path_list)}")
        for path in path_list:
            # Print paths using aliases for better readability
            print("Path:", " -> ".join(g_networkx.nodes[node]['alias'] for node in path))
    except nx.NodeNotFound as e:
        print(str(e))
    except Exception as e:
        print("An error occurred:", str(e))
else:
    print("Source or target not found in the graph.")


Total paths found: 4
Path: sorukumar -> WalletOfSatoshi.com -> satjar
Path: sorukumar -> ACINQ -> satjar
Path: sorukumar -> Kraken 🐙⚡ -> satjar
Path: sorukumar -> LQwD-Canada -> satjar


In [137]:
def find_path_with_constraints(graph, source_attr, target_attr, attribute, algorithm='dijkstra', excluded_nodes=None, max_length=6):
    """
    Find a path in a graph using the specified algorithm with node exclusions and maximum path length constraints.
    
    Args:
    - graph (nx.Graph): The graph object.
    - source_attr (str): The attribute value for the source node.
    - target_attr (str): The attribute value for the target node.
    - attribute (str): The attribute name used for node identification.
    - algorithm (str): 'dijkstra' or 'astar' for the algorithm choice.
    - excluded_nodes (list): List of node attributes to exclude from the path.
    - max_length (int): Maximum number of nodes in the path including source and target.
    
    Returns:
    - list: Path from source to target or an empty list if no path is found.
    """
    # Get source and target nodes based on attributes
    source = get_node_index_by_attribute(graph, attribute, source_attr)
    target = get_node_index_by_attribute(graph, attribute, target_attr)
    
    if source is None or target is None:
        return []  # Return empty if source or target nodes are not found
    
    # Create a copy of the graph to work with temporary modifications
    temp_graph = graph.copy()

    # Remove excluded nodes if any
    if excluded_nodes:
        excluded_nodes_ids = [get_node_index_by_attribute(graph, attribute, attr) for attr in excluded_nodes]
        excluded_nodes_ids = [node for node in excluded_nodes_ids if node is not None]
        temp_graph.remove_nodes_from(excluded_nodes_ids)

    # Try finding a path based on the chosen algorithm
    try:
        if algorithm == 'astar':
            # Define a heuristic for A*
            def heuristic(u, v):
                # Example heuristic: Absolute difference in node identifiers (modify as needed)
                return abs(int(u) - int(v))
            
            path = nx.astar_path(temp_graph, source, target, heuristic=heuristic, weight='weight')
        else:
            path = nx.dijkstra_path(temp_graph, source, target, weight='weight')
        
        # Check path length constraint
        if max_length and len(path) > max_length:
            return []  # Return empty if the path is too long
        
        return path

    except (nx.NetworkXNoPath, nx.NodeNotFound):
        # Return empty list if no path is found or nodes are not in graph
        return []

def print_path_with_aliases(graph, path, attribute):
    """Prints the path with node aliases."""
    id_to_alias = {node: attrs.get(attribute) for node, attrs in graph.nodes(data=True)}
    path_aliases = [id_to_alias.get(node, node) for node in path]
    return path_aliases



source_attr = '1027534202'  # Example source node attribute value
target_attr = '2120228994'  # Example target node attribute value
excluded_nodes = ['3157882931']  # Node attributes to exclude
max_path_length = 4  # Maximum nodes in the path
attribute_name = 'name'  # Attribute name used for node identification

# Find paths using both algorithms
path_dijkstra = find_path_with_constraints(g_networkx, source_attr, target_attr, attribute_name, 'dijkstra', excluded_nodes, max_path_length)
path_astar = find_path_with_constraints(g_networkx, source_attr, target_attr, attribute_name, 'astar', excluded_nodes, max_path_length)

# Print paths with aliases
print("Dijkstra's path with aliases:", print_path_with_aliases(g_networkx, path_dijkstra, 'alias'))
print("A* path with aliases:", print_path_with_aliases(g_networkx, path_astar, 'alias'))

Dijkstra's path with aliases: ['sorukumar', 'ACINQ', 'satjar']
A* path with aliases: ['sorukumar', 'ACINQ', 'satjar']


In [ ]:


def heuristic(node, target):
    # This is a simple heuristic that could be replaced with a more appropriate one.
    # For example, if you have spatial data, it could be the Euclidean distance.
    return 0

def a_star(graph, source_index, target_index):
    # A* pathfinding
    return graph.get_shortest_paths(
        source_index,
        to=target_index,
        weights='weight',  # Ensure your graph has a 'weight' attribute or change as needed
        output='vpath',
        mode=ig.OUT,
        algorithm="astar",
        astar_heuristic=lambda v, to: heuristic(graph.vs[v], graph.vs[to])
    )

# Usage
source_node = 'SourceNodeName'  # Replace with your source node name
target_node = 'TargetNodeName'  # Replace with your target node name
source_index = g.vs.find(name=source_node).index
target_index = g.vs.find(name=target_node).index

paths = a_star(g, source_index, target_index)
print([g.vs[vertex]['alias'] for vertex in paths[0]])


In [ ]:


def dijkstra(graph, source_index, target_index):
    # Dijkstra's algorithm for shortest path finding
    return graph.get_shortest_paths(
        source_index,
        to=target_index,
        weights='weight',  # This assumes your edges have a 'weight' attribute
        output='vpath',
        mode=ig.OUT
    )

# Usage
source_node = 'SourceNodeName'  # Replace with your source node name
target_node = 'TargetNodeName'  # Replace with your target node name
source_index = g.vs.find(name=source_node).index
target_index = g.vs.find(name=target_node).index

paths = dijkstra(g, source_index, target_index)
print([g.vs[vertex]['alias'] for vertex in paths[0]])


In [ ]:
# Assuming you have a graph 'g' and it has weights on the edges
source_node = '3334852914'  # Example source node name
target_node = '3836442376'  # Example target node name
source_index = g.vs.find(name=str(source_node)).index
target_index = g.vs.find(name=str(target_node)).index

# Find shortest path using Dijkstra's algorithm
path = g.get_shortest_paths(source_index, to=target_index, weights="weight", output="vpath")
path_vertices = [g.vs[vertex_index]['alias'] for vertex_index in path[0]]
print("Shortest path using Dijkstra's:", path_vertices)


In [ ]:
def heuristic(v, to, graph):
    # Simple heuristic: number of edges (could be replaced with a more sophisticated one)
    return abs(v.index - to.index)

# Ensure you pass the graph object and 'to' vertex to the heuristic
path = g.get_shortest_paths(
    source_index, 
    to=target_index, 
    weights='weight', 
    output='vpath', 
    algorithm="astar",
    astar_heuristic=lambda v, to: heuristic(g.vs[v], g.vs[to], g)
)
path_vertices = [g.vs[vertex_index]['alias'] for vertex_index in path[0]]
print("Shortest path using A*:", path_vertices)
